In [ ]:
from helpers import calculate_vector_field_eigenpairs
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_resolution=14)
print(f"randomFieldV.J: {randomFieldV.J}")
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_resolution=10)
print(f"randomFieldV.J: {randomFieldV.J}")

In [ ]:
import numpy as np
from helpers import *
import matplotlib.pyplot as plt

In [ ]:
class Quad_point():
    def __init__(self, point: list, weight: float):
        self.point = point
        self.weight = weight

QUAD_POINTS_2DD_6 = [Quad_point([(6 - np.sqrt(15)) / 21, (6 - np.sqrt(15)) / 21], (155 - np.sqrt(15)) / 2400),
                        Quad_point([(9 + 2 * np.sqrt(15)) / 21, (6 - np.sqrt(15)) / 21], (155 - np.sqrt(15)) / 2400),
                        Quad_point([(6 - np.sqrt(15)) / 21, (9 + 2 * np.sqrt(15)) / 21], (155 - np.sqrt(15)) / 2400),
                        Quad_point([(6 + np.sqrt(15)) / 21, (9 - 2 * np.sqrt(15)) / 21], (155 + np.sqrt(15)) / 2400),
                        Quad_point([(6 + np.sqrt(15)) / 21, (6 + np.sqrt(15)) / 21], (155 + np.sqrt(15)) / 2400),
                        Quad_point([(9 - 2 * np.sqrt(15)) / 21, (6 + np.sqrt(15)) / 21], (155 + np.sqrt(15)) / 2400),
                        Quad_point([1 / 3, 1 / 3], 9/80)]


# Create plot of the quad points
import matplotlib.pyplot as plt
import matplotlib.patches as patches

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
ax.set_aspect('equal', adjustable='datalim')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

for quad_point in QUAD_POINTS_2DD_6:
    ax.add_patch(patches.Circle(quad_point.point, 0.01, color='black'))

# Plot unit triangle
plt.plot([0, 1, 0], [0, 0, 1], 'black')
plt.plot([0, 1, 0, 0], [0, 0, 1, 0], 'black')
plt.xlabel('x', fontsize=24)
plt.ylabel('y', fontsize=24)
plt.title('2DD-6 quadrature rule', fontsize=24, pad=20)
ax.tick_params(axis='both', which='major', labelsize=16)



plt.show()


In [ ]:
from helpers import calculate_vector_field_eigenpairs
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(10)
xi = np.random.uniform(-np.sqrt(3), np.sqrt(3), randomFieldV.J)

In [ ]:
import numpy as np
import mshr
from helpers import DOMAIN, calculate_vector_field_eigenpairs
import fenics as fe
import matplotlib.pyplot as plt

mesh = mshr.generate_mesh(DOMAIN, 10)
# perturbed mesh based on the "original" mesh used for the KL-expansion
perturbed_coordinates = mesh.coordinates().copy()
for index, coordinate in enumerate(mesh.coordinates()):
    perturbed_coordinates[index] = randomFieldV(coordinate, xi)
# Create a new mesh with the perturbed coordinates
perturbed_mesh = fe.Mesh(mesh)
perturbed_mesh.coordinates()[:] = perturbed_coordinates

# Plot the original mesh and the perturbed mesh in one figure with different colors
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(111)
fe.plot(mesh, color='blue', linewidth=0.5, label='Original Mesh')
fe.plot(perturbed_mesh, color='red', linewidth=0.5, label='Perturbed Mesh')
plt.xlabel(r'$x_1$', fontsize=24)
plt.ylabel(r'$x_2$', fontsize=24)
ax.tick_params(axis='both', which='major', labelsize=16)
plt.legend(fontsize=16, markerscale=4)
# plt.title(f"Original and Perturbed Mesh")
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
plt.show()

In [ ]:
import mshr
import numpy as np
import matplotlib.pyplot as plt

mesh_coordinates = mshr.generate_mesh(DOMAIN, 32).coordinates()
random_field_values = np.array([randomFieldV(coord.copy(), xi) for coord in mesh_coordinates])
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
plt.scatter(mesh_coordinates[:, 0], mesh_coordinates[:, 1], color='blue', s=1, label='Mesh Points')
plt.quiver(mesh_coordinates[:, 0], mesh_coordinates[:, 1],
           random_field_values[:, 0] - mesh_coordinates[:, 0],
           random_field_values[:, 1] - mesh_coordinates[:, 1],
           color='red', scale=0.3, scale_units='xy', angles='xy', width=0.002, label='Random Field')
# plt.title("Random Field on Mesh Points")
plt.xlabel(r'$x_1$', fontsize=24)
plt.ylabel(r'$x_2$', fontsize=24)
plt.legend(fontsize=16, markerscale=4)
plt.axis('equal')
plt.tick_params(axis='both', which='major', labelsize=16)
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
plt.show()

In [ ]:
# Poisson û
import mshr
import numpy as np
import matplotlib.pyplot as plt
from helpers import solve_poisson_for_given_sample, RHS_F, DOMAIN
from scipy.interpolate import griddata
import plotly.graph_objects as go
from matplotlib.ticker import MaxNLocator


u_sol = solve_poisson_for_given_sample(10, jacobianV, xi, RHS_F)

mesh = mshr.generate_mesh(DOMAIN, 10)
x_coords = mesh.coordinates()[:, 0]
y_coords = mesh.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1:1:500j, -1:1:500j]
u_sols = []

for i in range(len(x_coords)):
    u_sols.append(u_sol(x_coords[i], y_coords[i]))

grid_z = griddata((x_coords, y_coords), u_sols, (grid_x, grid_y), method='linear')

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
cbar = plt.colorbar(cp)
cbar.ax.tick_params(labelsize=16)
cbar.locator = MaxNLocator(nbins=5)
cbar.update_ticks()
ax.set_xlim([-1.1, 1.1])
ax.set_ylim([-1.1, 1.1])
plt.title(r'$\hat{u}(\hat{x}, \omega)$ on $D_{ref}$', fontsize=24, pad=20)
plt.xlabel(r'$\hat{x}_1$', fontsize=24)
plt.ylabel(r'$\hat{x}_2$', fontsize=24)
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

In [ ]:
# Poisson u
import mshr
import numpy as np
import matplotlib.pyplot as plt
from helpers import solve_poisson_for_given_sample, RHS_F, DOMAIN, inverse_mapping
from scipy.interpolate import griddata
import plotly.graph_objects as go
import fenics as fe
from matplotlib.ticker import MaxNLocator


u_sol = solve_poisson_for_given_sample(10, jacobianV, xi, RHS_F)

mesh = mshr.generate_mesh(DOMAIN, 10)
x_coords = mesh.coordinates()[:, 0]
y_coords = mesh.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1.5:1.5:500j, -1.5:1.5:500j]
u_sols = []
x_coords_perturbed = []
y_coords_perturbed = []

for i in range(len(x_coords)):
    P_hat = fe.Point(x_coords[i], y_coords[i])
    x_coords_perturbed.append(randomFieldV(P_hat, xi)[0])
    y_coords_perturbed.append(randomFieldV(P_hat, xi)[1])
    u_sols.append(u_sol(P_hat))


grid_z = griddata((x_coords_perturbed, y_coords_perturbed), u_sols, (grid_x, grid_y), method='linear')

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
cbar = plt.colorbar(cp)
cbar.ax.tick_params(labelsize=16)
cbar.locator = MaxNLocator(nbins=5)
cbar.update_ticks()
ax.set_xlim([np.min(x_coords_perturbed) - 0.1, np.max(x_coords_perturbed) + 0.1])
ax.set_ylim([np.min(y_coords_perturbed) - 0.1, np.max(y_coords_perturbed) + 0.1])
plt.title(r'$u(x, \omega)$ on $D(\omega)$', fontsize=24, pad=20)
plt.xlabel(r'$x_1$', fontsize=24)
plt.ylabel(r'$x_2$', fontsize=24)
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

In [ ]:
# images: eigenvalues and eigenfunctions of random field KLE

# This cell takes 22 minutes

from helpers import *

mesh = mshr.generate_mesh(DOMAIN, 10)
V = fe.FunctionSpace(mesh, "DG", 0)
N = V.dim()

basis_functions = []
for i in range(N):
    basis_function = fe.Function(V)
    basis_function.vector()[i] = 1.0
    basis_function.set_allow_extrapolation(True)

    cell_index = V.dofmap().cell_dofs(i)[0]
    cell = fe.Cell(mesh, cell_index)
    vertex_coords = np.array([mesh.coordinates()[vertex] for vertex in cell.entities(0)])

    basis_functions.append(ConstBasisFunction(basis_function, vertex_coords))

C = np.zeros((2 * N, 2 * N))
for i, basis_function_i in enumerate(basis_functions):
    for j, basis_function_j in enumerate(basis_functions):
        if j <= i:
            # Here we use that each block is symmetric because of the symmetry of the covariance functions
            C[i, j] = C[j, i] = get_C_entry(v_cov1_1, basis_function_i, basis_function_j)
            C[i, N + j] = C[j, N + i] = get_C_entry(v_cov1_2, basis_function_i, basis_function_j)
            C[N + i, j] = C[N + j, i] = get_C_entry(v_cov2_1, basis_function_i, basis_function_j)
            C[N + i, N + j] = C[N + j, N + i] = get_C_entry(v_cov2_2, basis_function_i, basis_function_j)

M = np.zeros((2 * N, 2 * N))
for i, basis_function_i in enumerate(basis_functions):
    integrand = basis_function_i.function * basis_function_i.function * fe.dx
    M[i, i] = M[N + i, N + i] = fe.assemble(integrand)

J = 2*N # Number of eigenvectors -> J = N is maximum
eigenvalues, eigenvectors = eig(C, M)
eigenvalues = eigenvalues[:J].real
eigenvectors = eigenvectors[:, :J].real
sorted_indices = np.argsort(eigenvalues)[::-1]
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

# Eliminate negative eigenvalues
for index, sorted_eigenvalue in enumerate(sorted_eigenvalues):
    if sorted_eigenvalue < 0:
        sorted_eigenvalues[index] = 0

randomFieldV10 = RandomFieldV(sorted_eigenvalues, sorted_eigenvectors, basis_functions, N, J)
jacobianV10 = JacobianV(sorted_eigenvalues, sorted_eigenvectors, basis_functions, N, J)

In [ ]:
# Plot the eigenvalues called: eigenvalues
plt.figure(figsize=(10, 8))
plt.plot(randomFieldV10.eigenvalues[:N], 'o')
plt.xlabel('Eigenvalue Index', fontsize=24)
plt.ylabel('Eigenvalue', fontsize=24)
plt.yscale('log')
plt.tick_params(axis='both', which='major', labelsize=18)
plt.legend(fontsize=16, markerscale=4)
plt.show()

print(f"N: {N}")

In [ ]:
# Plot the eigenfunctions called: eigenfunctions_first_component and eigenfunctions_second_component
from matplotlib.ticker import FormatStrFormatter, MaxNLocator


eval_eigenfunc_k = lambda x, k: np.array([sum([randomFieldV.basis_functions[j].function(x) * randomFieldV.eigenvectors[j, k] for j in range(randomFieldV.N)]), 
                                          sum([randomFieldV.basis_functions[j].function(x) * randomFieldV.eigenvectors[j + randomFieldV.N, k] for j in range(randomFieldV.N)])])

x_coords = mesh.coordinates()[:, 0]
y_coords = mesh.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1:1:500j, -1:1:500j]

columns = 4
# First component
fig, axs = plt.subplots(1, columns, figsize=(16, 4))

for k in range(columns):
    ax = axs[k]
    z_values = []
    for i in range(len(x_coords)):
        z_values.append(eval_eigenfunc_k([x_coords[i], y_coords[i]], k)[0])
    
    grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')
    cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
    cbar = plt.colorbar(cp)
    cbar.ax.tick_params(labelsize=16)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()
    cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))  # Round to 2 decimal places
    ax.set_title(rf'$\gamma_{k+1}$', fontsize=24)
    if k == 0:
        ax.set_xlabel(r'$x_1$', fontsize=24)
        ax.set_ylabel(r'$x_2$', fontsize=24)
    else:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
    ax.tick_params(axis='both', which='major', labelsize=16)

plt.tight_layout()
plt.show()


# Second component
fig, axs = plt.subplots(1, columns, figsize=(16, 4))
for k in range(columns):
    ax = axs[k]
    z_values = []
    for i in range(len(x_coords)):
        z_values.append(eval_eigenfunc_k([x_coords[i], y_coords[i]], k)[1])
    
    grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')
    cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
    cbar = plt.colorbar(cp)
    cbar.ax.tick_params(labelsize=16)
    cbar.locator = MaxNLocator(nbins=5)
    cbar.update_ticks()
    cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))  # Round to 2 decimal places
    ax.set_title(rf'$\gamma_{k+1}$', fontsize=24)
    if k == 0:
        ax.set_xlabel(r'$x_1$', fontsize=24)
        ax.set_ylabel(r'$x_2$', fontsize=24)
    else:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
    ax.tick_params(axis='both', which='major', labelsize=16)
    

plt.tight_layout()
plt.show()

# Print first 4 eigenvalues
print(f"Eigenvalue 1: {randomFieldV.eigenvalues[0]}")
print(f"Eigenvalue 2: {randomFieldV.eigenvalues[1]}")
print(f"Eigenvalue 3: {randomFieldV.eigenvalues[2]}")
print(f"Eigenvalue 4: {randomFieldV.eigenvalues[3]}")

In [ ]:
# plots for poisson mc-analysis

from helpers import *

analyse_two_resolutions_from_data_u_hat(resolution_sparse = 10,
                                  resolution_fine = 14,
                                  P_hat = fe.Point(0.2, 0.2))

In [ ]:
# Sobol plot

from helpers import calculate_vector_field_eigenpairs, poisson_sobol_calc_indices_from_data, poisson_plot_sobols

mesh_res = 8
size_of_xi = 8
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_res)

In [ ]:
# import importlib
# importlib.reload(poisson_sobol_calc_indices_from_data)
# importlib.reload(poisson_plot_sobols)

S_single, S_total, mc_sample_size = poisson_sobol_calc_indices_from_data(fem_res=mesh_res, kl_res=mesh_res,
                                                                         size_of_xi=size_of_xi,
                                                                         randomFieldV=randomFieldV, jacobianV=jacobianV)
poisson_plot_sobols(S_single, S_total, mc_sample_size, title=f"Sobol Indices for Poisson Problem Mesh resolution {mesh_res}")

In [ ]:
import numpy as np
import mshr
from helpers import DOMAIN, calculate_vector_field_eigenpairs  # Assuming this is defined
import fenics as fe
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.ticker import MaxNLocator

# mesh_resolution = 8
# randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_resolution)
# xi = np.random.uniform(-np.sqrt(3), np.sqrt(3), randomFieldV.J)
# mesh = mshr.generate_mesh(DOMAIN, mesh_resolution)
# perturbed_coordinates = mesh.coordinates().copy()
# for index, coordinate in enumerate(mesh.coordinates()):
#     perturbed_coordinates[index] = randomFieldV(coordinate, xi)
# # Create a new mesh with the perturbed coordinates
# perturbed_mesh = fe.Mesh(mesh)
# perturbed_mesh.coordinates()[:] = perturbed_coordinates

fig, axs = plt.subplots(1, 2, figsize=(20, 8))

# Plotting (Corrected - Using fe.plot correctly)
ax1 = axs[0]
plt.subplot(1, 2, 1)
fe.plot(mesh, color='blue', linewidth=0.5, label=r'Reference Mesh $D_{ref}$')
ax1.set_xlabel(r'$x_1$', fontsize=30)
ax1.set_ylabel(r'$x_2$', fontsize=30)
ax1.legend(fontsize=30, markerscale=4, loc='upper left')
ax1.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax1.yaxis.set_major_locator(MaxNLocator(nbins=5))
ax1.tick_params(axis='both', which='major', labelsize=25)

ax2 = axs[1]
plt.subplot(1, 2, 2)
fe.plot(perturbed_mesh, color='red', linewidth=0.5, label=r'Perturbed Mesh $D(\omega)$')
ax2.set_xlabel(r'$x_1$', fontsize=30)
ax2.set_ylabel(r'$x_2$', fontsize=30)
ax2.legend(fontsize=30, loc='upper right')
ax2.tick_params(axis='both', which='major', labelsize=25)
ax2.yaxis.set_major_locator(MaxNLocator(nbins=5))
ax2.xaxis.set_major_locator(MaxNLocator(nbins=5))


# Add first arrow
xyA = [0.9, 0.6]
xyB = [-0.9, 0.6]

arrow = patches.ConnectionPatch(
    xyA,
    xyB,
    coordsA=ax1.transData,
    coordsB=ax2.transData,
    color="black",
    arrowstyle="-|>",
    mutation_scale=30,
    linewidth=3,
    connectionstyle="arc3,rad=-0.2"
)
fig.patches.append(arrow)

# Label first arrow
fig.text(0.51, 0.87, r'$V$', ha='center', va='center', fontsize=24)

# Add second arrow
xyA = [0.9, -0.6]
xyB = [-0.9, -0.6]

arrow = patches.ConnectionPatch(
    xyB,
    xyA,
    coordsA=ax2.transData,
    coordsB=ax1.transData,
    color="black",
    arrowstyle="-|>",
    mutation_scale=30,
    linewidth=3,
    connectionstyle="arc3,rad=-0.2"
)
fig.patches.append(arrow)

# Label second arrow
fig.text(0.51, 0.3, r'$V^{-1}$', ha='center', va='center', fontsize=24)

plt.tight_layout()
plt.show()
